# Code by TCE team at Qur'an QA 2023 shared task A

# Installation

I use [rclone](https://rclone.org/) to access my drive without asking for permission everytime.
The code accesses a file called colab4 which has my drive access token, you may replicate this on your side or just ignore this altogether and download files manually.  

In [1]:
!curl -O https://rclone.org/install.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4734  100  4734    0     0  13787      0 --:--:-- --:--:-- --:--:-- 14005


In [2]:
!chmod +x install.sh

'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
!sudo bash install.sh

'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

In [4]:
!rclone

'rclone' is not recognized as an internal or external command,
operable program or batch file.


## Clone repo and prepare the datasets

In [6]:
repo_url = f"https://github.com/Mohamedbasem1/Graduation-Project.git"
!git clone $repo_url


Cloning into 'quran-qa'...


In [5]:
!pip install -r requirements.txt
!pip install --no-deps python-terrier==0.7.1

     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.4 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 44.4/44.4 kB 437.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/112.7 kB ? eta -:--:--
     -------------- ------------------------ 41.0/112.7 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 112.7/112.7 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with sta

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\medob\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\medob\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [6]:
!pip install pwd


ERROR: Could not find a version that satisfies the requirement pwd (from versions: none)
ERROR: No matching distribution found for pwd

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\medob\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



## Cross Encoder

A cross encoder is a bert-based model that predicts a relevance score for a pair of sentences (late-interaction)

* We have different models to choose from the list below.
* Set the number of models to train, we train 10 models to get average performance.

* choose the experiment mode
    1.  QQA23_TaskA_qrcd_v1.2  ➡ normal training with official training data and validation with official validation data.  
    2.  QQA23_TaskA_qrcd_v1.2_merged ➡ combining training and validation for training and perform inference using hidden split (done for testing phase).
    3. tafseer  ➡ For tafseer pratraining data pairs
    4. pre-train ➡ For tydi-qa pratraining data pairs

[Check this for more details](https://www.sbert.net/examples/applications/cross-encoder/README.html)

In [9]:
import os
from random import choice
import glob
#deepset/roberta-base-squad2 , deepset/bert-medium-squad2-distilled
#Running : ahotrod/electra_large_discriminator_squad2_512
# next : deepset/electra-base-squad2
model_name = r"ahotrod/electra_large_discriminator_squad2_512"
num_models = 1 # @param {type:"integer"}

experiment_mode = "QQA23_TaskA_qrcd_v1.2"  # @param ["QQA23_TaskA_qrcd_v1.2", "QQA23_TaskA_qrcd_v1.2_merged","all_dev","pre-train","tafseer"]

lr = "1e-6"  # @param ["2e-5","1e-5","5e-6","2e-6","1e-6"]


for idx in range(num_models):
    out_file = f"{idx}-out.txt"
    err_file = f"{idx}-err.txt"
    doc_file="data/new_transeleted_quran.tsv"

    if experiment_mode == "QQA23_TaskA_qrcd_v1.2_merged":
        train_qrel_file = "data/QQA23_TaskA_qrels_merged.gold"
        train_query_file = "data/QQA23_TaskA_merged.tsv"
    elif experiment_mode == "QQA23_TaskA_qrcd_v1.2":
        # train_qrel_file = "data/praphrased_train.gold"
        train_query_file = "data/Transelted_train.tsv"
        train_qrel_file = "data/QQA23_TaskA_qrels_train.gold"
        # train_query_file = "data/QQA23_TaskA_train.tsv"
    elif experiment_mode == "all_dev":
        print("mode3")
        train_qrel_file = "data/QQA23_TaskA_qrels_dev.gold"
        train_query_file = "data/QQA23_TaskA_dev.tsv"

    validation_qrel_file = "data/QQA23_TaskA_qrels_dev.gold"
    validation_query_file = "data/Transeleted_dev.tsv"

  
    test_qrel_file = "data/QQA23_TaskA_qrels_test.gold"
    test_query_file = "data/Transeleted_test.tsv"
    num_train_epochs = 5
    pre_train = False
    do_predict = True
    do_eval= True

 


    if experiment_mode == "pre-train":
        doc_file="data/TYDI_QA_DOC.tsv"
        train_qrel_file = "data/TYDI_QA_qrels_train.gold"
        train_query_file = "data/TYDI_QA_train.tsv"
        validation_qrel_file = "data/TYDI_QA_qrels_dev.gold"
        validation_query_file = "data/TYDI_QA_dev.tsv"
        test_qrel_file = None
        test_query_file = None
        pre_train = True
        do_predict = False
        num_train_epochs = 2

    if experiment_mode == "tafseer":
        doc_file="data/tafseer_docs.tsv"
        train_qrel_file = "data/tafseer-qrel.tsv"
        train_query_file = "data/tafseer-query.tsv"
        validation_qrel_file = None
        validation_query_file = None
        test_qrel_file = None
        test_query_file = None
        pre_train = False
        do_eval= False
        do_predict = False
        num_train_epochs = 5


    output_folder = os.path.split(model_name)[-1] + f"-cross-v1.3-fine-tuned-{float(lr)}"

    batch_size = 4 


    !git pull
    !rm -r $output_folder

    !python "cross_encoder/trainer.py" \
        --model_name_or_path  "sultan/BioM-ELECTRA-Base-SQuAD2-BioASQ8B" \
        --do_train True \
        --do_eval $do_eval \
        --do_predict $do_predict \
        --train_qrel_file $train_qrel_file \
        --train_query_file  $train_query_file \
        --validation_qrel_file  $validation_qrel_file \
        --validation_query_file $validation_query_file \
        --test_qrel_file $test_qrel_file  \
        --test_query_file  $test_query_file \
        --doc_file $doc_file \
        --learning_rate $lr \
        --num_train_epochs $num_train_epochs \
        --max_seq_length 256 \
        --output_dir "$output_folder" \
        --per_device_eval_batch_size 4 \
        --per_device_train_batch_size 4 \
        --save_steps 2 \
        --overwrite_output_dir \
        --fp16 \
        --gradient_accumulation_steps 2


Already up to date.


'rm' is not recognized as an internal or external command,
operable program or batch file.
Traceback (most recent call last):
  File "c:\Users\medob\Desktop\Graduation Project\Quran\Quran QA 2023\Task-A\cross_encoder\trainer.py", line 6, in <module>
    from sentence_transformers import InputExample, CrossEncoder
  File "c:\Users\medob\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\__init__.py", line 10, in <module>
    from sentence_transformers.cross_encoder.CrossEncoder import CrossEncoder
  File "c:\Users\medob\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\__init__.py", line 3, in <module>
    from .CrossEncoder import CrossEncoder
  File "c:\Users\medob\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py", line 18, in <module>
    from sentence_transformers.evaluation.SentenceEvaluator import SentenceEvaluator
  File "c:\Users\medob\AppData\Loc

# Dual-encoder

A dual-encoder is a bert-based model that predicts a relevance score for a pair of sentences represented individually (representational-based).
The following cells trains ➡ infers ➡ mines hard negatives ➡ trains again.

* We have different models to choose from the list below.
* Set the number of models to train, we train 10 models to get average performance.

* choose the experiment mode
    1. QQA  ➡ normal training with official training data and validation with official validation data.  
    2. QQA-merged ➡ combining training and validation for training and perform inference using hidden split (done for testing phase).
    3. TYDI ➡ For tydi-qa pratraining data pairs

[Check DRhard repo for more details](https://github.com/jingtaozhan/DRhard)

In [7]:
!pip install sentence_transformers

   ---------------------------------------- 0.0/255.8 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/255.8 kB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 122.9/255.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 255.8/255.8 kB 2.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Analysis and ensemble

**Once the training is made you will find a dump file saved!**

**Once the training is made you will find a dump file saved!**

something like: araelectra-base-discriminator-tafseer-pairs-fine-tuned-1e-06-5254-train.zip
This is an araelectra-base-discriminator-tafseer-pairs fine-tuned model with:
1. learning rate of 1e-06.
2. A random starting seed of 5254.
4. train.zip means training data is used

This dump file contains models prediction for the given eval or test data.

You can look at the **analysis** directory of the repo for more details.
You can group dump files into folders:
1. run **performance_analysis.py** script to process and get results for single models and ensemble models
   - **retrieval_ensemble.py** is consumed by **performance_analysis.py** to implement the ensemble logic


In [21]:
!pwd
%cd quran-qa/Quran\ QA\ 2023/Task-A

/teamspace/studios/this_studio/quran-qa/Quran QA 2023/Task-A


[Errno 2] No such file or directory: 'quran-qa/Quran QA 2023/Task-A'
/teamspace/studios/this_studio/quran-qa/Quran QA 2023/Task-A


In [9]:
!python analysis/performance_analysis.py

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.12s/it]
artifacts/dumps 0.7648741075698899
